In [ ]:
!pip install tensorflow pandas matplotlib scikit-learn transformers tqdm nltk

In [ ]:
# import os
import pandas as pd
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
import numpy as np
from tqdm import tqdm
import nltk
# nltk.download('stopwords', quiet=True)
# nltk.download('wordnet', quiet=True)
# nltk.download('omw-1.4', quiet=True)
import string

import string
import re
df = pd.read_csv('/content/train.csv')
# df.head()
df.drop(columns=['id','toxic','severe_toxic','threat','insult','identity_hate'],inplace=True)
df.head()

In [ ]:
df[df['toxic']==1]['comment_text']

In [ ]:
df[(df['toxic'] == 1) & (df['obscene'] == 1)]

In [ ]:
df[df['obscene']==1]['comment_text']

In [ ]:
print([string.punctuation])
def remove_punctuation(message):
    punctuationfree="".join([i for i in message if i not in string.punctuation])
    return punctuationfree

df['comment_text']= df['comment_text'].apply(lambda x:remove_punctuation(x))
df.head()

['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']


,comment_text,obscene
0,Explanation\nWhy the edits made under my usern...,0
1,Daww He matches this background colour Im seem...,0
2,Hey man Im really not trying to edit war Its j...,0
3,\nMore\nI cant make any real suggestions on im...,0
4,You sir are my hero Any chance you remember wh...,0


In [ ]:
obscene_data= df[df['obscene']==1]

In [ ]:
X = df['comment_text'][:50000]
y = df['obscene'][:50000]
batch_size=16
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.cache()
dataset = dataset.shuffle(50000)
# dataset = dataset.batch(16)
# dataset = dataset.prefetch(8) # helps bottlenecks
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

# MAX_FEATURES = 200000 # number of words in the vocab



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length=256
def convert_example_to_feature(review):
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = max_length, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )

def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label


def encode_examples(ds, limit=-1):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []
  if (limit > 0):
      ds = ds.take(limit)

  for review, label in tqdm(ds):
    review = review.numpy().decode('utf-8') if isinstance(review.numpy(), bytes) else review
    label = label.numpy().decode('utf-8') if isinstance(label.numpy(), bytes) else label
    bert_input = convert_example_to_feature(review)
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([label])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# print(train)
ds_train_encoded = encode_examples(train).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(test).batch(batch_size)
# validation dataset
ds_val_encoded = encode_examples(val).batch(batch_size)

  0%|          | 0/35000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 10000/10000 [00:19<00:00, 501.68it/s]


In [ ]:
learning_rate = 2e-5
# we will do just 1 epoch, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1

# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [ ]:
# ds_test_encoded
import gc
gc.collect()

18

In [ ]:
model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)

2188/2188 [==============================] - 2105s 944ms/step - loss: 0.0611 - accuracy: 0.9780 - val_loss: 0.0458 - val_accuracy: 0.9790


In [ ]:
def pre_processing_data(input_text):
    input_text="".join([i for i in input_text if i not in string.punctuation])
    input_text= input_text.split(' ')
    print(input_text)
    return convert_example_to_feature(input_text)
input= pre_processing_data("hello, i'm good you T#ell your St0tus as programmer.")
input

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['hello', 'im', 'good', 'you', 'Tell', 'your', 'St0tus', 'as', 'programmer']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': [101, 7592, 10047, 2204, 2017, 100, 2115, 100, 2004, 20273, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:

input_text = 'You freaking suck! I am going to hit you.'

def prediction(input_text):
  predict_input = tokenizer.encode(input_text,truncation=True,padding=True,return_tensors="tf")
  res = model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(res, axis=1)
  label = tf.argmax(tf_prediction, axis=1) #labels = {0:'clear', 1:'obscene'}
  return label.numpy()[0]


In [ ]:
x=tokenizer.encode(input_text,truncation=True,padding=True,return_tensors="tf")
x

<tf.Tensor: shape=(1, 13), dtype=int32, numpy=
array([[  101,  2017, 13847, 11891,   999,  1045,  2572,  2183,  2000,
         2718,  2017,  1012,   102]], dtype=int32)>

In [ ]:
input_text = 'you like f8cKer.'
print(prediction(input_text))

In [ ]:
input_text = 'f@cking looser.'
print(prediction(input_text))
input_text = 'fucking looser.'
print(prediction(input_text))
input_text = 'you like porn movies.'
print(prediction(input_text))
input_text = 'do you like p@rnmovies.'
print(prediction(input_text))
input_text = 'you scum. pathetic cUnt'
print(prediction(input_text))
input_text = 'fck you.'
print(prediction(input_text))
input_text = 'scumbag kill you'
print(prediction(input_text))
input_text = 'it okey buddy.nothing to worry.'
print(prediction(input_text))

1/1 [==============================] - 3s 3s/step
1
1/1 [==============================] - 0s 37ms/step
1
1/1 [==============================] - 0s 38ms/step
0
1/1 [==============================] - 0s 41ms/step
0
1/1 [==============================] - 0s 37ms/step
1
1/1 [==============================] - 0s 48ms/step
1
1/1 [==============================] - 0s 40ms/step
1
1/1 [==============================] - 0s 36ms/step
0


In [ ]:
results = model.predict(ds_test_encoded)[0]
results


313/313 [==============================] - 105s 328ms/step


array([[ 1.5334551, -1.9761088],
       [ 2.970151 , -3.5454364],
       [ 1.8804033, -2.391343 ],
       ...,
       [-1.2772477,  0.4636533],
       [ 4.0583773, -3.9534519],
       [ 3.837812 , -4.042522 ]], dtype=float32)

In [ ]:
tf_prediction = tf.nn.softmax(results, axis=1)
labels = ['clear','obscene'] #(0:negative, 1:positive)
label = tf.argmax(tf_prediction, axis=1)

In [ ]:
# from sklearn.metrics import precision_score, recall_score, accuracy_score
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()
true_Y=[]
for batch in tqdm(ds_test_encoded.as_numpy_iterator()):
    # Unpack the batch
    X_true, y_true = batch
    # Make a prediction
    # result = model.predict(X_true)[0]
    # tf_prediction = tf.nn.softmax(result)
    # labels = ['Negative','Positive'] #(0:negative, 1:positive)
    # yhat = tf.argmax(tf_prediction)

    # Flatten the predictions
    y_true = y_true.flatten()
    # yhat = yhat.flatten()
    true_Y.extend(y_true)
    # print(yhat,y_true)
print(true_Y)
print(label)
pre.update_state(true_Y,label)
re.update_state(true_Y, label)
acc.update_state(true_Y, label)
print(f'Precision: {pre.result()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

313it [00:00, 1719.79it/s]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
tokenizer.save_pretrained('model/tokenizer/')
model.save_pretrained('model/model/')